In [3]:
boston_data = pd.read_csv("https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/BostonHousing.csv")

data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

In [4]:
dizionario_nomi = {
    'crim': 'tasso_criminalità',
    'zn': 'proporzione_terreno_residenziale',
    'indus': 'proporzione_acri_non_commerciali',
    'chas': 'confine_fiume_charles',
    'nox': 'concentrazione_ossidi_azoto',
    'rm': 'numero_medio_stanze',
    'age': 'proporzione_unità_occupate_costruite_prima_1940',
    'dis': 'distanza_centri_lavoro',
    'rad': 'indice_accessibilità_autostrade',
    'tax': 'tasso_imposta_proprietà',
    'ptratio': 'rapporto_alunni_insegnanti',
    'b': 'proporzione_b',
    'lstat': 'percentuale_stato_socioeconomico_inferiore',
    'medv': 'valore_medio_case'
}



In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari



# Inizializzazione dell'app Dash con il tema Material
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])


# Inizializzazione dei componenti di input
xaxis_feature_dropdown = dcc.Dropdown(
    id='xaxis-feature',
    options=[{'label': dizionario_nomi[col], 'value': col} for col in boston_data.columns],
    value='rm'  # Valore predefinito
)

yaxis_feature_dropdown = dcc.Dropdown(
    id='yaxis-feature',
    options=[{'label': dizionario_nomi[col], 'value': col} for col in boston_data.columns],
    value='medv'  # Valore predefinito
)

hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{'label': dizionario_nomi[col], 'value': col} for col in boston_data.columns],
    value='medv'  # Valore predefinito
)

violin_feature_dropdown = dcc.Dropdown(
    id='violin-feature',
    options=[{'label': dizionario_nomi[col], 'value': col} for col in boston_data.columns],
    value='medv'  # Valore predefinito
)

variabili_checklist = dbc.Checklist(
    id='correlation-checklist',
    options=[{'label': dizionario_nomi[col], 'value': col} for col in boston_data.columns],
    value=['crim', 'medv', 'lstat'],  # Valori predefiniti
    inline=True
)

# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi dei Dati delle Abitazioni di Boston"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse X:"),
              xaxis_feature_dropdown,
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              yaxis_feature_dropdown,
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              hist_feature_dropdown,
            ])
        , width=6, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='scatter-plot')
              ])
            , width=6),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        , width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a violin plot:"),
                violin_feature_dropdown,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              variabili_checklist,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='violin-plot')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='heatmap')
            ])
        ], width=8),
    ]),
])

# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('xaxis-feature', 'value'),
     Input('yaxis-feature', 'value')]
)
def update_scatter(xaxis_feature, yaxis_feature):
    fig = px.scatter(boston_data, x=xaxis_feature, y=yaxis_feature, color='medv',
                     title=f'Grafico a dispersione di {dizionario_nomi[yaxis_feature]} vs {dizionario_nomi[xaxis_feature]}',
                     labels={xaxis_feature: xaxis_feature, yaxis_feature: yaxis_feature})
    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50),)
    return fig

# Callback per l'istogramma
@app.callback(
    Output('histogram', 'figure'),
    [Input('hist-feature', 'value')]
)
def update_histogram(selected_feature):
    # Crea un istogramma basato sulla caratteristica selezionata
    fig = px.histogram(boston_data, x=selected_feature,
                       color_discrete_sequence=['#007bff'],  # Imposta il colore su blu
                       opacity=0.7,  # Imposta l'opacità per una migliore visibilità
                       title=f'Istogramma di {selected_feature}',
                       labels={selected_feature: selected_feature})

    # Personalizza il layout
    fig.update_layout(bargap=0.01,  # Imposta lo spazio tra le barre
                      bargroupgap=0.05,  # Imposta lo spazio tra i gruppi di barre
                      xaxis_title=selected_feature,  # Imposta il titolo dell'asse x
                      yaxis_title='Conteggio',  # Imposta il titolo dell'asse y
                      showlegend=False,  # Nasconde la legenda
                      margin=dict(l=0, r=0, t=50, b=30))  # Rimuove i margini

    return fig

# Callback per violin plot
@app.callback(
    Output('violin-plot', 'figure'),
    [Input('violin-feature', 'value')]
)
def update_violin_plot(selected_feature):
    # Crea violin plot sulla feature selezionata
    fig = px.violin(boston_data, y=selected_feature, box=True,
                    title=f'Violin Plot of {selected_feature}',
                    labels={selected_feature: dizionario_nomi[selected_feature]})
    fig.update_layout(margin=dict(l=30, r=30, t=70, b=30))  # Update layout to remove margins
    return fig

# Callback per la heatmap
@app.callback(
    Output('heatmap', 'figure'),
    [Input('correlation-checklist', 'value')]
)
def update_heatmap(selected_features):
    # Seleziona le colonne corrispondenti alle variabili selezionate
    selected_data = boston_data[selected_features]

    # Calcola la correlazione tra le variabili selezionate
    correlation_matrix = selected_data.corr()

    # Crea la heatmap
    fig = px.imshow(correlation_matrix,
                    labels=dict(color="Correlazione"),
                    x=correlation_matrix.index,
                    y=correlation_matrix.columns,
                    color_continuous_scale='RdBu',
                    zmin=-1, zmax=1)

    fig.update_layout(title="Heatmap Correlazione tra Variabili Selezionate",
                      xaxis_showgrid=False,
                      yaxis_showgrid=False)

    fig.update_traces(hoverongaps=False,
                      hovertemplate='Variabile X: %{y}<br>Variabile Y: %{x}<br>Correlazione: %{z:.2f}')

    return fig

# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")





Dash app running on http://127.0.0.1:8050/
